# Modified Gale Shapley Algorithm for Generative Data Comparison

## Set-Up
We'll primarily using pandas to work with the data. We start off by importing pandas and importing the data as a pandas dataframe.

In [23]:
import pandas as pd
import numpy as np
import numpy as np
import matplotlib.pyplot as plt

In [24]:
syndata = pd.read_csv('synData9.csv')
realdata = pd.read_csv('kag_risk_factors_cervical_cancer.csv')

In [ ]:
syndata.iloc[0] #Use this to access a row
syndata.loc[0, : ] #Use this to access a row
x = syndata.loc[0,"Age"] #Use this to access a column
syndata.columns[0]

## Data Cleaning and Helper Functions
We need to clean up the data before we use it. Primarily, this means we will need to do some normalization to the data.

One important thing we may need later on is the means for each of the columns. First, we compute mean for the raw data.

In [25]:
def mean_for_column(data1, column):
    """
    Finds the mean for a specific column
    """
    tot = 0 
    for i in range(len(data1)):
        #print(data1.loc[i, column])
        if data1.loc[i, column] == "?" or data1.loc[i, column] == -1:
            pass
        else:
            tot = tot + float(data1.loc[i, column])
    tot = tot/(len(data1))
    return tot
    
def mean_for_cols(data1):
    """
    Computes the mean for all the columns 
    """
    ar = []
    for col in data1.columns: 
        ar.append(mean_for_column(data1, col))
    return ar
    
def clean(data1):
    """
    Replaces the missing data with a -1 value and converts the strings into floats
    """
    for i in range(len(data1)):
        for f in data1.columns:
            #print(data1.loc[i, f])
            if data1.loc[i, f] == "?":
                data1.loc[i, f] = float(-1)
                continue
            if isinstance(data1.loc[i, f], str):
                data1.loc[i, f] = float(data1.loc[i, f])
    data1 =data1.set_axis([f for f in range(len(data1))], axis='index')
    #print(data1.dtypes)
    return data1.astype('float')
    
def norm(data1):
    """
    Min-Max normalizes the data
    """
    new_dat = data1.copy()
    for col in new_dat.columns:
        new_dat[col] = (new_dat[col] - new_dat[col].min()) / (new_dat[col].max() - new_dat[col].min())
    return new_dat

In [35]:
syndata_means = mean_for_cols(syndata)
realdata_means = mean_for_cols(realdata)

#Back up of the data
syndata_safe = syndata 
realdata_safe = realdata

realdata_clean = clean(realdata)
syndata_clean = clean(syndata)

realdata = realdata_safe
syndata = syndata_safe


#For syndata
syndata_clean_norm = norm(syndata_clean)
#For realdata
realdata_clean_norm = norm(realdata_clean)

syndata_clean_norm = syndata_clean_norm.fillna(-1)
realdata_clean_norm = realdata_clean_norm.fillna(-1)
#They both should be False, this is important
syndata_clean_norm.isnull().values.any(), realdata_clean_norm.isnull().values.any()

(False, False)

## Weights

In [36]:
#We start of by getting the column names
#print(realdata_clean_norm.columns)
#print(syndata_clean_norm.columns)

In [37]:
#They're the same so we use 1 single column array
cols = ['Age', 'Number of sexual partners', 'First sexual intercourse',
       'Num of pregnancies', 'Smokes', 'Smokes (years)', 'Smokes (packs/year)',
       'Hormonal Contraceptives', 'Hormonal Contraceptives (years)', 'IUD',
       'IUD (years)', 'STDs', 'STDs (number)', 'STDs:condylomatosis',
       'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis',
       'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis',
       'STDs:pelvic inflammatory disease', 'STDs:genital herpes',
       'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:HIV',
       'STDs:Hepatitis B', 'STDs:HPV', 'STDs: Number of diagnosis',
       'STDs: Time since first diagnosis', 'STDs: Time since last diagnosis',
       'Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller',
       'Citology', 'Biopsy']

#Init
real_weights =[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
    1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
syn_weights =[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
    1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
test_weights =[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
    1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

high_weights =['Age', 'Number of sexual partners', 'First sexual intercourse',
       'Num of pregnancies', 'Smokes']
med_weights =['Smokes (years)', 'Smokes (packs/year)',
       'Hormonal Contraceptives', 'Hormonal Contraceptives (years)', 'IUD',
       'IUD (years)', 'STDs', 'STDs (number)', 'STDs:condylomatosis',
       'STDs:cervical condylomatosis']
low_weights =['STDs:vaginal condylomatosis',
       'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis',
       'STDs:pelvic inflammatory disease', 'STDs:genital herpes',
       'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:HIV',
       'STDs:Hepatitis B', 'STDs:HPV', 'STDs: Number of diagnosis',
       'STDs: Time since first diagnosis', 'STDs: Time since last diagnosis',
       'Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller',
       'Citology', 'Biopsy']

for i in range(len(cols)):
    if cols[i] in high_weights:
        real_weights[i] = 3
        syn_weights[i] = 3
    if cols[i] in med_weights:
        real_weights[i] = 2
        syn_weights[i] = 2
    if cols[i] in low_weights:
        real_weights[i] = 1
        syn_weights[i] = 1

## Comparison Function
The comparison function is the main function that compares the data. It compares all the synthetic data to one specific row of the real data and creates an array of the most similar matches.

We also compute the values without using the functions.

In [34]:
def compare(data1, data2, row_comp, weights, mini=False):
    """
    This is a comparsion function  
    data1: First dataset for comparsion, 1 set row
    data2: Second dataset for comparsion
    row_comp: The row we want compared
    mini: Chooses if the function returns the minimum value or not
    """
    #Goes for 1 section of the real data and then loops for every row of the synthesized data
    lst = []
    for i in range(len(data2)):
        #print('Real data: ' + str(data1.loc[row_comp, "Age"]) + '. Fake data: ' +str(data2.loc[i, 'Age']))
        #Comparsion function to compare each the set real row to current fake row
        lst.append(similiar(data1, data2, i, row_comp, weights))
    if mini:
        lst = min(lst)
    return lst

def similiar(data1, data2, ind, row_comp, weights):
    """
    Computes how similiar the two datasets are 
    """
    score = 0
    for i in range(len(data1.columns)):
        score = score + (abs(data1.loc[row_comp,:][i] - data2.loc[ind,:][i]) * weights[i])
        #print(data1.loc[row_comp,:] - data2.loc[ind,:])
    return score 

def compNew(data1, data2, row_comp, weights):
    lst = []
    
    for i in range(len(data2)):
        score = 0
        for y in range(len(data1.columns)):
            score = score + (abs(data1.loc[row_comp,:][y] - data2.loc[i,:][y]) * weights[y])
        lst.append(score)
    return lst

In [ ]:
#Data Splitting
syndata_clean_norm_split = np.array_split(syndata_clean_norm, 100)
realdata_clean_norm_split = np.array_split(realdata_clean_norm, 17)
realsplit1 = realdata_clean_norm_split[0]
synsplit1 = syndata_clean_norm_split[0]
realsplit1 = realsplit1.drop(50)
len(realsplit1), len(synsplit1)

In [ ]:
synsplit1

In [ ]:
xasda = compNew(synsplit1, realsplit1, 0, test_weights)

In [ ]:
len(xasda)

In [ ]:
synD = [str(i+1) for i in range(50)]
realD = [str(i+1) for i in range(50)]

In [ ]:
heatscores=[]

for i in range(len(realsplit1)):
    heatscores.append(compNew(synsplit1, realsplit1, i, test_weights))


In [ ]:

fig, ax = plt.subplots()
im = ax.imshow(heatscores)



# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(synD)), labels=synD)
ax.set_yticks(np.arange(len(realD)), labels=realD)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")



ax.set_title("Trial")
plt.colorbar()
plt.show()




In [ ]:
compare(realdata_clean_norm,syndata_clean_norm, 0, test_weights)

Calling each columns with [,:] and then indexing into it with [i]
The columns are being called already but the values within the columns are not being called. That is what this i is doing


We don't need to compare element by element, we can do a vector comparison. 


.* for element by element for 2 vectors


Or use matrix multiplication (might need to transpose weights)




In [ ]:
compare_arr = compare(realdata_clean_norm,syndata_clean_norm, 0, test_weights)

In [ ]:
#big_comp = []

In [ ]:
for i in range(len(realdata_clean_norm)):
    x = compare(realdata_clean_norm,syndata_clean_norm, i, test_weights)
    big_comp.append(x)

#Took 511m 43.3s .....

In [ ]:
len(big_comp)

In [ ]:
import numpy as np

arr = np.array(big_comp)
index_values = []
for i in range(858):
    index_values.append(i)
column_values = []
for i in range(5000):
    column_values.append(i)

In [ ]:
df = pd.DataFrame(data = arr, 
                  index = index_values, 
                  columns = column_values)

In [ ]:
df.to_csv('data.csv', index=True)

In [ ]:
ival = []
for i in range(5000):
    ival.append(i)

In [ ]:
min(compare_arr)
compare_arr.index(18.114858335405216)
syndata_clean.iloc[509, :]

In [ ]:
"""
#How to access the number from a specific column and row
syndata_clean_norm.loc[0,:"Age"][0]
#Goes through each column iteritvely of the synthetic data row that's closest with the first row of the real data
for i in range(36):
    print("Fake: " + str(syndata.iloc[509][i]) + ". Real: " + str(realdata.iloc[0][i]))
#How to find the minimium of the compare_arr and figure out the location 
min(compare_arr)
compare_arr.index(18.114858335405216)
"""

In [ ]:
score = 0
for i in range(len(syndata_clean_norm.columns)):
    #print(realdata_clean_naorm.loc[0,:])
    score = score + ((realdata_clean_norm.loc[0,:][i] - syndata_clean_norm.loc[0,:][i])) #* weights[i])

sy = [12,2,12,3,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
re = [18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1,-1,0,0,0,0,0,0,0,0]
ar = [syndata_clean_norm.loc[0,:][i] for i in range(len(syndata.columns))]
ar2 = [realdata_clean_norm.loc[0,:][i] for i in range(len(syndata.columns))]

sum = 0
for i in range(len(sy)):
    sum = sum + abs(ar2[i] - ar[i])
    
assert sum == compare_arr[0], "Not working"

## Ranking
Now that we've made the comparison function, we need a function that creates a threshold that decides if the data is similar enough

In [ ]:
def Average(l):
    result = 0
    for i in range(len(l)):
        result = result + l[0]
    result = result/len(l)
    return result

In [ ]:
compare_arr_avg = Average(compare_arr)
compare_arr_avg, min(compare_arr), max(compare_arr)